# Market Catalog API

This notebook demonstrates the `fetch_market_catalog()` method for browsing and filtering available markets.

The market catalog contains metadata about all available prediction markets including:
- Market IDs and descriptions
- Assets (BTC, ETH, SOL, etc.)
- Horizons (daily, weekly, monthly, yearly)
- Market types (up-or-down, above, below)
- Time ranges and expiration dates

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv("../.env.local")

from polybridge import PolybridgeClient
from datetime import datetime, timedelta, timezone
import pandas as pd

In [ ]:
client = PolybridgeClient(api_key=os.getenv("POLYBRIDGE_API_KEY"))

## Example 1: Basic Catalog Query

Fetch all available markets (no filters):

In [ ]:
catalog = client.fetch_market_catalog()

markets = catalog["markets"]
print(f"Total markets available: {len(markets)}")
print(f"\nFirst market:")
print(markets[0])

## Example 2: Convert to DataFrame

Working with catalog data as a pandas DataFrame:

In [ ]:
catalog_df = pd.DataFrame(markets)
print(f"Shape: {catalog_df.shape}")
print(f"\nColumns: {catalog_df.columns.tolist()}")
print(f"\nFirst few rows:")
catalog_df.head()

## Example 3: Filter by Asset

Get markets for specific cryptocurrencies:

In [ ]:
btc_catalog = client.fetch_market_catalog(
    assets=["BTC"]
)

btc_markets = btc_catalog["markets"]
print(f"BTC markets: {len(btc_markets)}")

# Show breakdown by market type
btc_df = pd.DataFrame(btc_markets)
print(f"\nBTC markets by type:")
print(btc_df["market_type"].value_counts())

## Example 4: Filter by Multiple Criteria

Combine multiple filters:

In [ ]:
filtered_catalog = client.fetch_market_catalog(
    assets=["BTC", "ETH"],
    horizons=["daily", "weekly"],
    market_types=["up-or-down"]
)

filtered_markets = filtered_catalog["markets"]
print(f"Filtered markets: {len(filtered_markets)}")

# Group by asset and horizon
filtered_df = pd.DataFrame(filtered_markets)
print(f"\nBreakdown by asset and horizon:")
print(filtered_df.groupby(["asset", "horizon"]).size())

## Example 5: Analyze Catalog Statistics

Get overview statistics from the full catalog:

In [ ]:
# Get full catalog
full_catalog = client.fetch_market_catalog()
df = pd.DataFrame(full_catalog["markets"])

print("=" * 60)
print("CATALOG STATISTICS")
print("=" * 60)

print(f"\nTotal Markets: {len(df)}")

print(f"\nAssets:")
print(df["asset"].value_counts())

print(f"\nHorizons:")
print(df["horizon"].value_counts())

print(f"\nMarket Types:")
print(df["market_type"].value_counts())

print(f"\nMarkets by Asset and Horizon:")
pivot = pd.crosstab(df["asset"], df["horizon"])
print(pivot)

## Example 6: Filter by Time Range

Find markets active during a specific time period:

In [ ]:
# Get markets active in the last 7 days
end = datetime.now(timezone.utc)
start = end - timedelta(days=7)

time_filtered = client.fetch_market_catalog(
    assets=["BTC"],
    start_ts=start.isoformat(),
    end_ts=end.isoformat(),
)

time_markets = time_filtered["markets"]
print(f"Markets active in last 7 days: {len(time_markets)}")
print(f"\nSample markets:")
pd.DataFrame(time_markets)[["market_id", "market_type", "horizon"]].head(10)

## Example 7: Extract Market IDs for Low-Level APIs

Use the catalog to get market IDs for low-level API calls:

In [ ]:
# Get BTC daily up-or-down markets
catalog = client.fetch_market_catalog(
    assets=["BTC"],
    horizons=["daily"],
    market_types=["up-or-down"],
)

market_ids = [m["market_id"] for m in catalog["markets"]]
print(f"Found {len(market_ids)} market IDs")
print(f"\nFirst 5 IDs:")
for mid in market_ids[:5]:
    print(f"  - {mid}")

print(f"\nThese IDs can be used with:")
print("  - client.fetch_probabilities(markets=market_ids, ...)")
print("  - client.fetch_prices(markets=market_ids, ...)")
print("  - client.fetch_options_metrics(markets=market_ids, ...)")

## Example 8: Compare Market Coverage Across Assets

See which assets have the most market coverage:

In [ ]:
import matplotlib.pyplot as plt

# Get full catalog
catalog = client.fetch_market_catalog()
df = pd.DataFrame(catalog["markets"])

# Count markets by asset and type
coverage = df.groupby(["asset", "market_type"]).size().unstack(fill_value=0)

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
coverage.plot(kind="bar", stacked=True, ax=ax)
ax.set_title("Market Coverage by Asset and Type")
ax.set_xlabel("Asset")
ax.set_ylabel("Number of Markets")
ax.legend(title="Market Type")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nMarket counts:")
print(coverage)

## Example 9: Search for Specific Markets

Find markets matching specific criteria:

In [ ]:
# Get all markets
catalog = client.fetch_market_catalog()
df = pd.DataFrame(catalog["markets"])

# Example: Find all "above" markets for SOL
sol_above = df[
    (df["asset"] == "SOL") & 
    (df["market_type"] == "above")
]

print(f"Found {len(sol_above)} SOL 'above' markets")
print(f"\nMarkets:")
sol_above[["market_id", "horizon", "market_type"]].head(10)

## Example 10: Export Catalog to CSV

Save the catalog for external analysis:

In [ ]:
# Get filtered catalog
catalog = client.fetch_market_catalog(
    assets=["BTC", "ETH"],
    horizons=["daily"],
)

df = pd.DataFrame(catalog["markets"])

# Export to CSV
output_file = "market_catalog_btc_eth_daily.csv"
df.to_csv(output_file, index=False)
print(f"Exported {len(df)} markets to {output_file}")
print(f"\nColumns exported: {df.columns.tolist()}")

## Summary

The `fetch_market_catalog()` method is essential for:

1. **Discovery** - Browse all available markets
2. **Filtering** - Find markets matching specific criteria
3. **Analysis** - Understand market coverage and availability
4. **Integration** - Extract market IDs for use with low-level APIs

**Available Filters:**
- `assets` - Filter by cryptocurrency (BTC, ETH, SOL, etc.)
- `horizons` - Filter by time horizon (daily, weekly, monthly, yearly)
- `market_types` - Filter by market type (up-or-down, above, below)
- `start_ts` / `end_ts` - Filter by time range

The catalog is the starting point for most data queries in the Polybridge API!